In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

# Fashion MNIST 이미지 분류하기
인공신경망으로 이미지를 분류해봅시다!!
---

In [4]:
USE_CUDA = torch. cuda.is_available() #GPU를 연결합니다.
DEVICE = torch.device("cuda" if USE_CUDA else "cpu") #GPU를 쓸수있으면 키고 안되면 CPU

In [5]:
# 하이퍼 파라미터 설정
EPOCHS = 30
BATCH_SIZE = 64

In [6]:
#텐서로 바꾸기 위한 변수
transform = transforms.Compose([
    transforms.ToTensor()
])

In [7]:
#데이터 불러오기
trainset = datasets.FashionMNIST(
    root = './.data/',
    train = True,
    download = True,
    transform = transform)

testset = datasets.FashionMNIST(
    root = './.data/',
    train = False,
    download = True,
    transform = transform)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./.data/FashionMNIST\raw\train-images-idx3-ubyte.gz to ./.data/FashionMNIST\raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./.data/FashionMNIST\raw\train-labels-idx1-ubyte.gz to ./.data/FashionMNIST\raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./.data/FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ./.data/FashionMNIST\raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./.data/FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ./.data/FashionMNIST\raw



In [8]:
#데이터를 나눠주기!!
train_loader = torch.utils.data.DataLoader(
    dataset = trainset,
    batch_size = BATCH_SIZE,
    shuffle = True)
test_loader = torch.utils.data.DataLoader(
    dataset = testset,
    batch_size = BATCH_SIZE,
    shuffle = True)

In [10]:
#인공신경망 만들기
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(784,256) #28*28의 이미지이므로 784개의 픽셀 정보를 받아온다.
        self.fc2 = nn.Linear(256,128)
        self.fc3 = nn.Linear(128,10) #10개의 종류로 분류하므로 마지막 레이어의 노드는 10개로 만든다.
    def forward(self,x):
        x = x.view(-1,784) #이미지 형태를 바꾸고
        x = F.relu(self.fc1(x)) #레이어를 이으며 엑티베이션을 해준다.
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [11]:
model = Net().to(DEVICE) #모델을 생성한다.
optimizer = optim.SGD(model.parameters(),lr = 0.01) #역전파를 위한 옵티마이저를 설정한다.

In [12]:
def train(model, train_loader, optimizer): #학습을 위한 함수를 만들어 준다.
    model.train() #학습 상태로 전환
    for batch_idx, (data,target) in enumerate(train_loader): #train_loader에 정보를 이용해 반복한다.
        data, target = data.to(DEVICE),target.to(DEVICE) #데이터들을 학습을 돌릴 디바이스에 넣는다.
        optimizer.zero_grad() #반복마다 기울기를 새로 설정하기에 zero_grad를 설정해준다.
        output = model(data) # 모델에 입력한다
        loss = F.cross_entropy(output,target) #분류 오차를 변수에 넣고
        loss.backward() #미분해서
        optimizer.step() # 학습률 만큼 역전파시킨다.
        

In [14]:
def evaluate(model, test_loader): #모델 실행 모드 정의이다.
    model.eval() #실행 모드로 바꾸고
    test_loss = 0 #오차와
    correct =0 # 정확도 변수를 만든다.
    with torch.no_grad(): # 실행 과정에서 기울기는 계산할 필요가 없다.
        for data,target in test_loader: # test_loader에 정보만큼 반복한다.
            data, target = data.to(DEVICE),target.to(DEVICE) #정보를 디바이스에 전달하고
            output = model(data) # 모델에 입력한다
            
            #학습하면서 생기는 오차들을 합한다
            test_loss +=F.cross_entropy(output,target, reduction='sum').item()
            pred = output.max(1,keepdim=True)[1] #가장 큰 값을 가지는 것이 예측값
            correct += pred.eq(target.view_as(pred)).sum().item() #예측한게 맞으면 correct에 더해주기
    #오차값의 합을 전체 데이터셋으로 나눠주고
    test_loss/=len(test_loader.dataset)
    
    #맞춘 정답의 갯수의 합을 전체 데이터셋으로 나누고 100을 곱해 백분율로 나타내기
    test_accuracy = 100.*correct/len(test_loader.dataset)
    return test_loss, test_accuracy

In [16]:
#학습 돌리기!!
for epoch in range(1,EPOCHS+1):
    train(model,train_loader,optimizer)
    test_loss, test_accuracy = evaluate(model,test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch, test_loss, test_accuracy))

[1] Test Loss: 0.8613, Accuracy: 67.37%
[2] Test Loss: 0.6627, Accuracy: 76.24%
[3] Test Loss: 0.5883, Accuracy: 78.84%
[4] Test Loss: 0.5568, Accuracy: 79.66%
[5] Test Loss: 0.5112, Accuracy: 81.75%
[6] Test Loss: 0.5251, Accuracy: 80.99%
[7] Test Loss: 0.4891, Accuracy: 82.89%
[8] Test Loss: 0.5024, Accuracy: 81.75%
[9] Test Loss: 0.4889, Accuracy: 82.76%
[10] Test Loss: 0.4712, Accuracy: 83.38%
[11] Test Loss: 0.4551, Accuracy: 84.03%
[12] Test Loss: 0.4378, Accuracy: 84.58%
[13] Test Loss: 0.4503, Accuracy: 84.08%
[14] Test Loss: 0.4436, Accuracy: 84.37%
[15] Test Loss: 0.4338, Accuracy: 84.66%
[16] Test Loss: 0.4554, Accuracy: 84.02%
[17] Test Loss: 0.4137, Accuracy: 85.33%
[18] Test Loss: 0.4280, Accuracy: 85.10%
[19] Test Loss: 0.4096, Accuracy: 85.67%
[20] Test Loss: 0.4039, Accuracy: 85.75%
[21] Test Loss: 0.3997, Accuracy: 85.94%
[22] Test Loss: 0.4286, Accuracy: 84.47%
[23] Test Loss: 0.4161, Accuracy: 85.25%
[24] Test Loss: 0.3988, Accuracy: 85.76%
[25] Test Loss: 0.4073, A